# Omnibot Coefficient Determination with Sympy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

### Using the symbolic formulas to generate the numeric model

In [ ]:
def generic_omnibot_mats(n = 4, null_beta = True, equal_r = True):
    t, r, d, s = symbols('t r d s')
    alpha, beta= dynamicsymbols('alpha beta')
    
    W = Matrix([
        [r, cos(alpha)],
        [0, sin(alpha)]
    ])
    T = Matrix([
        [cos(beta), -sin(beta)],
        [sin(beta), cos(beta)]
    ])
    A = Matrix([
        [1, 0, -d],
        [0, 1, s]
    ])
    WTA= W.inv()@T.inv()@A
    WTA.simplify()
    r_n = WTA[0,:]
    s_n = WTA[1,:]
    
    R_list = []
    S_list = []
    
    for ii in range(n):
        r_ii = r_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        r_ii = r_ii.subs(d, symbols('d_'+str(ii+1)))
        r_ii = r_ii.subs(s, symbols('s_'+str(ii+1)))
        s_ii = s_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        s_ii = s_ii.subs(d, symbols('d_'+str(ii+1)))
        s_ii = s_ii.subs(s, symbols('s_'+str(ii+1)))
        if not equal_r:
            r_ii = r_ii.subs(r, symbols('r_'+str(ii+1)))
            s_ii = s_ii.subs(r, symbols('r_'+str(ii+1)))
        if null_beta:
            r_ii = r_ii.subs(beta, 0)
            s_ii = s_ii.subs(beta, 0)
        else:
            r_ii = r_ii.subs(beta, symbols('beta_'+str(ii+1)))
            s_ii = s_ii.subs(beta, symbols('beta_'+str(ii+1)))
            
        S_list.append(s_ii)
        R_list.append(r_ii)
    
    R = Matrix(R_list)
    S = Matrix(S_list)
    return R, S

In [ ]:
def integerize(expr):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a))
    return expr2

def roundize(expr,n = 4):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a,n))
    return expr2

In [ ]:
def dejabot_mats():
    R, S = generic_omnibot_mats()
    L, l = symbols('L l')
    for ii in range(4):
        alpha = pi/4 * (1 - 2 * ((int((ii+1)/2))%2))
        s = L * (1 - 2 * ((int((ii)/2))%2))
        d = l * (1 - 2 * (ii%2))
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return integerize(R), integerize(S)

In [ ]:
t, r, d, s, m, I_z, I_w, l, L = symbols('t r d s m I_z I_w l L')
x, y, alpha, beta, sigma, psi, theta= dynamicsymbols('x y alpha beta sigma psi theta')
psi_dot = psi.diff()
x, y, alpha, beta, sigma, psi, psi_dot, theta, t, r, d, s, m, I_z, I_w, l, L

In [ ]:
q = [x, y, psi] + [dynamicsymbols(f'phi_{i+1}') for i in range(4)]
q = Matrix(q)
Gamma = [dynamicsymbols(f'tau_{i+1}') for i in range(4)]
Gamma = Matrix(Gamma)

q, Gamma

In [ ]:
M_w = eye(4)*I_w
M_r = Matrix([
    [m, 0, 0],
    [0, m, 0],
    [0, 0, I_z]
])
R_psi = Matrix([
    [cos(psi), -sin(psi), 0],
    [sin(psi), cos(psi), 0],
    [0, 0, 1]
])
R, S = dejabot_mats()

M_r, M_w, R_psi, R

In [ ]:
H = M_r + R_psi@R.T@M_w@R@R_psi.T
H = simplify(H)
K = R_psi@R.T@M_w@R@R_psi.diff().T
K = simplify(K)
F_a = R_psi@R.T@Gamma
F_a_0 = F_a[0].factor(sin(psi), cos(psi))
F_a_1 = F_a[1].factor(sin(psi), cos(psi))
F_a_2 = F_a[2].factor()
F_a = Matrix([F_a_0, F_a_1, F_a_2])
A = R_psi@R.T
A_inv = R@R_psi.T
H_inv = simplify(H.inv())
H, K, F_a,

In [ ]:
get_h_raw = lambdify([m, I_w, I_z, l, L, r], H)
get_k_raw = lambdify([I_w, r, psi_dot], K)
get_a_raw = lambdify([psi, l, L, r], A)
get_a_inv_raw = lambdify([psi, l, L, r], A_inv)
get_h_inv_raw = lambdify([m, I_w, I_z, l, L, r], H_inv)

## Electric Motor Model

$$ V = K_m \dot{\phi} + Ri$$
$$ \tau_m = K_ei - \tau_r$$

In [ ]:
def friction(a, b, phi_dot, V, eps_phi = 0.1):
    fric_wet = a * phi_dot
    fric_dry = np.where(npor(
        phi_dot>eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V > 0)
    ),b, 0)
    fric_dry = np.where(npor(
        phi_dot< -eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V < 0)
    ),-b, fric_dry)
    return fric_wet + fric_dry

In [ ]:
def create_motor_model(i_7000 = 0.5):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    t_r = -t_rat + K_e * i_rat
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}, t_r = {t_r}')
    def motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motor_model(phi_dot, V = 24):
        i = (V - K_m*phi_dot)/R
        t = K_e * i - friction(0,t_r, phi_dot, V)
        return t
    return motor_model, simp_motor_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,7000])
ax1.set_xlim([0,4400])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
plt.grid()
plt.title('Motor Curves')

plt.grid(True)

### Comparison with manufacturer original motor curves:
![Original curves](images_dejabot/curvas_motor.png)

Expanded curves outside normal regime:

In [ ]:
x = np.linspace(-2 * np.pi * 7000/60, 4 * np.pi * 7000/60, 300)
n = x * 30 / np.pi
motor_model = create_motor_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n, 'r')
ax1.set_ylim([-14000,14000])
ax1.set_xlim([-4400,8800])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([-26,26])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([-80,80])

ax4 = ax1.twinx()
ax4.plot(t,ef, 'g')
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([-2,2])
plt.grid()

plt.grid(True)

### Adapted Curves with reductor gears:

In [ ]:
def create_motoreduct_model(i_7000 = 0.5, N = 49, eff = 0.6, a = 0.01, b = 0.4):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}')
    eps_phi = 0.1
    
    def motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motoreduct_model(phi_dot, V = 24):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - friction(a, b, phi_dot, V, eps_phi)
        return t
    return motoreduct_model, simp_motoreduct_model

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/(49*60), 300)
n = x * 30 / np.pi
motor_model = create_motoreduct_model(0.5)[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,150])
ax1.set_xlim([0,130000])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
#plt.grid()

ax5 = ax1.twinx()
ax5.spines["right"].set_position(("axes", -0.2))
ax5.set_ylim([0, x[-1]*0.0667])

plt.grid(True)

In [ ]:
def torq_lims(phi_dot, v_max = 24, motor = create_motoreduct_model()[1]):
    t_max = motor(phi_dot, v_max)
    t_min = motor(phi_dot, -v_max)
    
    return t_max, t_min

In [ ]:
def vect_torq_lims(geom, q, q_dot):
    a = get_a(geom, q)
    phi_dot = a.T@q_dot
    #print(f'phi_dot: {np.round(phi_dot,3)}')
    t_max = []
    t_min = []
    for wheel in phi_dot:
        maxt, mint = torq_lims(wheel)
        t_max.append(maxt)
        t_min.append(mint)
    t_max = np.array(t_max)
    t_min = np.array(t_min)
    
    return (t_max, t_min)

# New motor model: parameter determination

When we are testing the motors in order to get the parameters, we can safely assume that $\dot{\phi}$ is going to be positive, and therefore we can simplify the friction to
$$ \tau_f = (1 + cF_n)(a\dot{\phi} + b) $$

In [ ]:
a, b, c, Fn, Km, Ke, i, R, tau, eta, N, V = symbols('a b c F_n K_m K_e i R tau eta N V')
phi= dynamicsymbols('phi')
phi_dot = phi.diff()
a, b, c, Fn, Km, Ke, i, R, tau, eta, N, V, phi_dot

In [ ]:
tau_f = (1 + c * Fn) * (a*phi_dot + b)
tau_f

In [ ]:
tau_m = eta * N * Ke * i
tau_m

In [ ]:
expr_v = N*Km*phi_dot + R*i
expr_v

In [ ]:
eq_1 = Eq(V, expr_v)
eq_1

In [ ]:
eq_2 = Eq(tau_m, tau_f)
eq_2

In [ ]:
i_eq_2 = solve(eq_2, i)[0]
i_eq_2

In [ ]:
i_eq_2

In [ ]:
eq_def = eq_1.subs(i, i_eq_2)
eq_def

In [ ]:
phi_dot_test = solve(eq_def, phi_dot)[0]
phi_dot_test

In [ ]:
phi_dot_test_fun = lambdify([a, b, c, Fn, V, R, Ke, Km, eta, N], phi_dot_test)

In [ ]:
def new_friction(a, b, c, phi_dot, V, F_n = 0, eps_phi = 0.1):
    fric_wet = a * phi_dot
    fric_dry = np.where(npor(
        phi_dot>eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V > 0)
    ),b, 0)
    fric_dry = np.where(npor(
        phi_dot< -eps_phi,
        npand(
            np.abs(phi_dot)<=eps_phi,
            V < 0)
    ),-b, fric_dry)
    return (1 + c * F_n) * (fric_wet + fric_dry)

In [ ]:
def create_motoreduct_model(i_7000 = 0.5, N = 49, eff = 0.6, a_0 = 0.01, b_0 = 0.4, c_0 = 0.1):
    V = 24
    R = 24/13
    K_m = (V - R*i_7000)/(2 * np.pi*7000/60)
    t_rat = 4300*9.8/(100*1000)    #570*9.8/(100*1000)
    phi_rat = 0                    #5900*np.pi*2/60
    i_rat = (V - K_m*phi_rat)/R
    K_e = t_rat/(i_rat-i_7000)
    
    print(f'K_m = {K_m}, K_e = {K_e}, R = {R}, i_rat = {i_rat}')
    eps_phi = 0.1
    
    def motoreduct_model(phi_dot, V = 24, F_n = 0, a = a_0, b = b_0, c = c_0):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - new_friction(a, b, c, phi_dot, V, F_n, eps_phi)
        p = t * phi_dot
        ef = p / (i * V)
        return t, i, p, ef
    def simp_motoreduct_model(phi_dot, V = 24, F_n = 0, a = a_0, b = b_0, c = c_0):
        phi_dot_motor = phi_dot * N
        i = (V - K_m*phi_dot_motor)/R
        t = K_e * i * N * eff - new_friction(a, b, c, phi_dot, V, F_n, eps_phi)
        return t    
    def phi_dot_test(V = 24, F_n = 0, a = a_0, b = b_0, c = c_0):
        phi_dot = phi_dot_test_fun(a, b, c, F_n, V, R, K_e, K_m, eff, N)
        tau_fric = new_friction(a, b, c, phi_dot, V, F_n, eps_phi)
        return phi_dot, tau_fric
    return motoreduct_model, simp_motoreduct_model, phi_dot_test

In [ ]:
x = np.linspace(0, 2 * np.pi * 7000/(49*60), 300)
n = x * 30 / np.pi
motor_model = create_motoreduct_model()[0]
t, i, p, ef = motor_model(x)
t = t *100000/9.8

fig, ax1 = plt.subplots()

ax1.plot(t,n)
ax1.set_ylim([0,150])
ax1.set_xlim([0,130000])

ax2 = ax1.twinx()
ax2.plot(t,i)
ax2.set_ylim([0,13])

ax3 = ax1.twinx()
ax3.plot(t,p)
ax3.spines["right"].set_position(("axes", 1.1))
ax3.set_ylim([0,80])

ax4 = ax1.twinx()
ax4.plot(t,ef)
ax4.spines["right"].set_position(("axes", 1.2))
ax4.set_ylim([0,1])
#plt.grid()

ax5 = ax1.twinx()
ax5.spines["right"].set_position(("axes", -0.2))
ax5.set_ylim([0, x[-1]*0.0667])

plt.grid(True)

In [ ]:
phi_test = create_motoreduct_model()[2]

In [ ]:
phi_test()

In [ ]:
motor_model(phi_test()[0])

### Expected test behaviour

In [ ]:
v_array = np.linspace(1, 24, 20)
phi_array, fric_array = phi_test(v_array)
plt.plot(phi_array, fric_array, marker = 'x')

plt.ylim([0,1])
plt.xlim([0,15])

## Parameter encoding in functions:
$$Geom = [l, L, r]$$
$$Mass = [m, I_w,I_z]$$
$$q = [x, y, \psi]$$ where q is $q_r$

In [ ]:
def get_h(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    return get_h_raw(m, I_w, I_z, l, L, r)

def get_k(geom, mass, q_dot):
    l, L, r = geom
    m, I_w, I_z = mass
    x_dot, y_dot, psi_dot = q_dot
    return get_k_raw(I_w, r, psi_dot)

def get_a(geom, q):
    l, L, r = geom
    x, y, psi = q
    return get_a_raw(psi, l, L, r)

def get_h_inv(geom, mass):
    l, L, r = geom
    m, I_w, I_z = mass
    return get_h_inv_raw(m, I_w, I_z, l, L, r)


In [ ]:
geom = [0.2096, 0.2096, 0.0667]
mass = [15.75, 0.00266, 0.461]
q_0 = np.array([0, 0, 0])
q_dot_0 = np.array([0, 0, 0])
vect_torq_lims(geom, q_0, q_dot_0+[0.9,0,0])